In [5]:
pip install --upgrade pip

In [1]:
!pip install neuralprophet

In [6]:
if 'google.colab' in str(get_ipython()):
    ! pip install git+https://github.com/ourownstory/neural_prophet.git

  Cloning https://github.com/ourownstory/neural_prophet.git to /tmp/pip-req-build-48ns5zxn
  Running command git clone --filter=blob:none --quiet https://github.com/ourownstory/neural_prophet.git /tmp/pip-req-build-48ns5zxn
  Resolved https://github.com/ourownstory/neural_prophet.git to commit ba50436b108d6d5b61071674a4972bf9f5b57241
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import pandas as pd
from neuralprophet import NeuralProphet
train = pd.read_csv('/content/drive/MyDrive/seoul/train.csv')
sub = pd.read_csv('/content/drive/MyDrive/seoul/sample_submission.csv')

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [8]:
train['일시'] = pd.to_datetime(train['일시'])
train.info()
# 일사합 : 년간 일사량(태양의 복사 에너지가 땅에 닿는 양)의 합계
# 일조합 : 년간 일사시간(구름이나 안개 따위에 가려지지 아니하고 햇볕이 실제로 내리쬐는 시간)의 합계
# 일조율 : 일조시간과 가조시간과의 비로, 1에 가까울수록 구름이 적은 경우가 된다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      23011 non-null  datetime64[ns]
 1   최고기온    23008 non-null  float64       
 2   최저기온    23008 non-null  float64       
 3   일교차     23007 non-null  float64       
 4   강수량     9150 non-null   float64       
 5   평균습도    23011 non-null  float64       
 6   평균풍속    23007 non-null  float64       
 7   일조합     22893 non-null  float64       
 8   일사합     18149 non-null  float64       
 9   일조율     22645 non-null  float64       
 10  평균기온    23011 non-null  float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 1.9 MB


In [12]:
sol = pd.read_csv('/content/drive/MyDrive/seoul/test.csv',encoding = 'cp949')
additional_dates = pd.date_range(start='2023-12-13', end='2023-12-24', freq='D')
additional_temperatures = [6.8, 7.6, 4.9, -4.1, -9.8, -7.7, -3.7, -7.4, -11.7, -11.6, -6.8, -2.2]

# Create a DataFrame from the additional data
new_sol = pd.DataFrame({'날짜': additional_dates, '평균기온(℃)': additional_temperatures})
sol2 = pd.concat([sol,new_sol],axis=0,ignore_index=True)
sol2.tail()

,날짜,평균기온(℃)
353,2023-12-20 00:00:00,-7.4
354,2023-12-21 00:00:00,-11.7
355,2023-12-22 00:00:00,-11.6
356,2023-12-23 00:00:00,-6.8
357,2023-12-24 00:00:00,-2.2


In [9]:
simple = train[['일시','평균기온']]
simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)
model = NeuralProphet()
loss = model.fit(simple, freq="D")

WARNING - (py.warnings._showwarnmsg) - <ipython-input-9-77ceb8904af5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to maj

Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

In [10]:
loss

,MAE,RMSE,Loss,RegLoss,epoch
0,13.547484,16.675873,0.146288,0.0,0
1,3.554153,4.439969,0.014138,0.0,1
2,2.532147,3.270007,0.007224,0.0,2
3,2.508840,3.244984,0.007101,0.0,3
4,2.494990,3.232197,0.007035,0.0,4
...,...,...,...,...,...
78,2.455359,3.187173,0.006846,0.0,78
79,2.453444,3.182192,0.006839,0.0,79
80,2.453179,3.184745,0.006838,0.0,80
81,2.453116,3.184395,0.006836,0.0,81


In [11]:
future_data = model.make_future_dataframe(df = simple,periods = 358) #periods는 예측할 기간
forecast_data = model.predict(future_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is

Predicting: 180it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [13]:
forecast_data.head()

,ds,y,yhat1,trend,season_yearly,season_weekly
0,2023-01-01,None,-1.284215,13.483032,-14.851953,0.084703
1,2023-01-02,None,-1.505590,13.483175,-14.937318,-0.051448
2,2023-01-03,None,-1.599229,13.483316,-15.017044,-0.065501
3,2023-01-04,None,-1.675344,13.483459,-15.090954,-0.067849
4,2023-01-05,None,-1.674282,13.483601,-15.158836,0.000953


In [16]:
pred = forecast_data[forecast_data['ds'].dt.year == 2023]['yhat1'].reset_index(drop=True)
sol2['pred'] = pred

sol2['eval'] = abs(sol2['평균기온(℃)'] - sol2['pred'])
sol2['eval'].mean()

2.6799309544057155

In [17]:
#test
# train['일시'] = pd.to_datetime(train['일시'])
simple = train[['일시','평균기온']]
simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)
prophet = NeuralProphet(
    n_changepoints=2170,  # changepoints의 수 설정
    ar_layers = 939,
    yearly_seasonality=True,  # 연간 계절성 사용 여부
    loss_func='mae',  # MAE 손실 함수 설정
)
prophet.fit(simple)

WARNING - (py.warnings._showwarnmsg) - <ipython-input-17-913147503a8f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically def

Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

,MAE,RMSE,Loss,RegLoss,epoch
0,50.383816,59.080463,0.832943,0.0,0
1,17.326164,22.185076,0.252933,0.0,1
2,8.794164,12.686174,0.123918,0.0,2
3,5.143074,7.404356,0.076665,0.0,3
4,3.670694,5.056180,0.058026,0.0,4
...,...,...,...,...,...
78,2.418283,3.152575,0.042073,0.0,78
79,2.417969,3.151868,0.042073,0.0,79
80,2.418225,3.153843,0.042081,0.0,80
81,2.415474,3.149112,0.042023,0.0,81


In [18]:
future_data = prophet.make_future_dataframe(df = simple,periods = 358) #periods는 예측할 기간
forecast_data = prophet.predict(future_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is

Predicting: 180it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [19]:
pred = forecast_data[forecast_data['ds'].dt.year == 2023]['yhat1'].reset_index(drop=True)
sol2['pred'] = pred

sol2['eval'] = abs(sol2['평균기온(℃)'] - sol2['pred'])
sol2['eval'].mean()

2.653356701834908

#test3

In [20]:
simple = train[['일시','평균기온']]
simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)
prophet = NeuralProphet(
    n_changepoints=10,  # changepoints의 수 설정
    yearly_seasonality=23,  # 연간 계절성 사용 여부
    loss_func='mae',  # MAE 손실 함수 설정
)
prophet.fit(simple)

WARNING - (py.warnings._showwarnmsg) - <ipython-input-20-982d273916b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically def

Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

,MAE,RMSE,Loss,RegLoss,epoch
0,16.683838,23.273367,0.290195,0.0,0
1,3.319521,4.498658,0.057655,0.0,1
2,2.577299,3.340730,0.044664,0.0,2
3,2.567283,3.339345,0.044527,0.0,3
4,2.598130,3.371433,0.045061,0.0,4
...,...,...,...,...,...
78,2.449998,3.190160,0.042583,0.0,78
79,2.451008,3.193686,0.042603,0.0,79
80,2.449850,3.188465,0.042571,0.0,80
81,2.446748,3.184559,0.042538,0.0,81


In [21]:
future_data = prophet.make_future_dataframe(df = simple,periods = 358) #periods는 예측할 기간
forecast_data = prophet.predict(future_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is

Predicting: 180it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [22]:
pred = forecast_data[forecast_data['ds'].dt.year == 2023]['yhat1'].reset_index(drop=True)
sol2['pred'] = pred

sol2['eval'] = abs(sol2['평균기온(℃)'] - sol2['pred'])
sol2['eval'].mean()

2.6555727596389516

In [ ]:
NeuralProphet()

In [92]:
prophet = NeuralProphet(

growth='off',
n_changepoints=10,
yearly_seasonality=True, #년간 계절성 설정
weekly_seasonality=False, #주간 계절성 설정
daily_seasonality=False, #일간 계절성 설정
batch_size=64,#배치 사이즈 설정
)

In [93]:
simple = train[['일시','평균기온']]
simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

prophet.fit(simple,freq='D',epochs=200)

WARNING - (py.warnings._showwarnmsg) - <ipython-input-93-84db97870a1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple.rename(columns={'일시': 'ds','평균기온': 'y'}, inplace=True)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to ma

Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

,MAE,RMSE,Loss,RegLoss,epoch
0,10.130999,13.376301,0.118110,0.0,0
1,2.511368,3.253843,0.007146,0.0,1
2,2.517127,3.254830,0.007164,0.0,2
3,2.523302,3.263609,0.007205,0.0,3
4,2.527231,3.271699,0.007234,0.0,4
...,...,...,...,...,...
195,2.498082,3.235907,0.007078,0.0,195
196,2.497683,3.233877,0.007076,0.0,196
197,2.498057,3.233639,0.007071,0.0,197
198,2.497914,3.235966,0.007077,0.0,198


In [96]:
future_data = prophet.make_future_dataframe(df = simple,periods = 358) #periods는 예측할 기간
forecast_data = prophet.predict(future_data)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.721% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.721% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is

Predicting: 360it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [97]:
pred = forecast_data[forecast_data['ds'].dt.year == 2023]['yhat1'].reset_index(drop=True)
sol2['pred'] = pred

sol2['eval'] = abs(sol2['평균기온(℃)'] - sol2['pred'])
sol2['eval'].mean()

2.978537814710393

In [28]:
pred = forecast_data[forecast_data['ds'].dt.year == 2023]['yhat1'].reset_index(drop=True)
sub['평균기온'] = pred
sub.head()

,일시,평균기온
0,2023-01-01,-1.352948
1,2023-01-02,NaN
2,2023-01-03,NaN
3,2023-01-04,NaN
4,2023-01-05,NaN
